# Data downloading

In [0]:
!git clone https://github.com/CyrilShch/IFCNN

Cloning into 'IFCNN'...
remote: Enumerating objects: 123, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (107/107), done.
remote: Total 433 (delta 32), reused 96 (delta 12), pack-reused 310
Receiving objects: 100% (433/433), 62.20 MiB | 27.96 MiB/s, done.
Resolving deltas: 100% (43/43), done.


In [0]:
# Detect if we are in Google Colaboratory
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

from pathlib import Path
# Determine the locations of auxiliary libraries and datasets.
# `AUX_DATA_ROOT` is where 'notmnist.py', 'animation.py' and 'tiny-imagenet-2020.zip' are.
if IN_COLAB:
    google.colab.drive.mount("/content/drive")
    
    # Change this if you created the shortcut in a different location
    AUX_DATA_ROOT = Path("/content/drive/My Drive/")
    
    assert AUX_DATA_ROOT.is_dir(), "Have you forgotten to 'Add a shortcut to Drive'?"
    
    import sys
    sys.path.insert(0, str(AUX_DATA_ROOT))
else:
    AUX_DATA_ROOT = Path(".")

# unzip the data
![ ! -d "Data" ] && unzip -q "{AUX_DATA_ROOT / 'wt_cropped_data_720x720'}"
![ ! -d "Ground_truth" ] && unzip -q "{AUX_DATA_ROOT / 'wt_cropped_gt_720x720'}"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
from collections import Counter

def get_idx_and_num_samples(root):
    data = [int(file.split("_")[0]) for files in os.walk(root) for file in files[2]]
    counter = dict(Counter(data))

    n = [key for key, _ in counter.items()]
    count = [value for _, value in counter.items()]

    return n, count

# IFCNN model


In [0]:
import os
import cv2
import time
import torch

from torchvision import transforms
from torch.autograd import Variable

from PIL import Image
import numpy as np

from IFCNN.Code.utils.myTransforms import denorm, norms, detransformcv2

In [0]:
'''
------------------------------------------------------------------------------
IFCNN: A General Image Fusion Framework Based on Convolutional Neural Network
------------------------------------------------------------------------------
'''
import torch
import math
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models


# My Convolution Block
class ConvBlock(nn.Module):
    def __init__(self, inplane, outplane):
        super(ConvBlock, self).__init__()
        self.padding = (1, 1, 1, 1)
        self.conv = nn.Conv2d(inplane, outplane, kernel_size=3, padding=0, stride=1, bias=False)
        self.bn = nn.BatchNorm2d(outplane)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        out = F.pad(x, self.padding, 'replicate')
        out = self.conv(out)
        out = self.bn(out)
        out = self.relu(out)
        return out

    
class IFCNN(nn.Module):
    def __init__(self, resnet, fuse_scheme=0):
        super(IFCNN, self).__init__()
        self.fuse_scheme = fuse_scheme # MAX, MEAN, SUM
        self.conv2 = ConvBlock(64, 64)
        self.conv3 = ConvBlock(64, 64)
        self.conv4 = nn.Conv2d(64, 3, kernel_size=1, padding=0, stride=1, bias=True)

        # Initialize parameters for other parameters
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))

        # Initialize conv1 with the pretrained resnet101 and freeze its parameters
        for p in resnet.parameters():
            p.requires_grad = False

        self.conv1 = resnet.conv1
        self.conv1.stride = 1
        self.conv1.padding = (0, 0)

    def tensor_max(self, tensors):
        max_tensor = None
        for i, tensor in enumerate(tensors):
            if i == 0:
                max_tensor = tensor
            else:
                max_tensor = torch.max(max_tensor, tensor)
        return max_tensor

    def tensor_sum(self, tensors):
        sum_tensor = None
        for i, tensor in enumerate(tensors):
            if i == 0:
                sum_tensor = tensor
            else:
                sum_tensor = sum_tensor + tensor
        return sum_tensor

    def tensor_mean(self, tensors):
        sum_tensor = None
        for i, tensor in enumerate(tensors):
            if i == 0:
                sum_tensor = tensor
            else:
                sum_tensor = sum_tensor + tensor
        mean_tensor = sum_tensor / len(tensors)
        return mean_tensor

    def operate(self, operator, tensors):
        out_tensors = []
        for tensor in tensors:
            out_tensor = operator(tensor)
            out_tensors.append(out_tensor)
        return out_tensors

    def tensor_padding(self, tensors, padding=(1, 1, 1, 1), mode='constant', value=0):
        out_tensors = []
        for tensor in tensors:
            out_tensor = F.pad(tensor, padding, mode=mode, value=value)
            out_tensors.append(out_tensor)
        return out_tensors

    def forward(self, *tensors):
        # Feature extraction
        outs = self.tensor_padding(tensors=tensors, padding=(3, 3, 3, 3), mode='replicate')
        outs = self.operate(self.conv1, outs)
        outs = self.operate(self.conv2, outs)
        
        # Feature fusion
        if self.fuse_scheme == 0: # MAX
            out = self.tensor_max(outs)
        elif self.fuse_scheme == 1: # SUM
            out = self.tensor_sum(outs)
        elif self.fuse_scheme == 2: # MEAN
            out = self.tensor_mean(outs)
        else: # Default: MAX
            out = self.tensor_max(outs)
        
        # Feature reconstruction
        out = self.conv3(out)
        out = self.conv4(out)
        return out


def myIFCNN(fuse_scheme=0):
    # pretrained resnet101
    resnet = models.resnet101(pretrained=True)
    # our model
    model = IFCNN(resnet, fuse_scheme=fuse_scheme)

    return model

In [0]:
import os
import torch
import torch.utils.data as data
from torchvision import datasets, transforms
from PIL import Image

IMG_EXTENSIONS = [
    '.jpg', '.JPG', '.jpeg', '.JPEG',
    '.png', '.PNG', '.ppm', '.PPM', '.bmp', '.BMP',
    '.tif'
    ]

class ImageSequence(data.Dataset):
    def __init__(self, is_folder=False, mode='RGB', transform=None, *impaths):
        self.is_folder = is_folder
        self.mode = mode
        self.transform = transform
        self.impaths = impaths

    def loader(self, path):
        return Image.open(path).convert(self.mode)

    def get_imseq(self):
        if self.is_folder:
            folder_path = self.impaths[0]
            impaths = self.make_dataset(folder_path)
        else:
            impaths = self.impaths

        imseq = []
        for impath in impaths:
            if os.path.exists(impath):
                im = self.loader(impath)
                if self.transform is not None:
                    im = self.transform(im)
                imseq.append(im)
        return imseq

    def is_image_file(self, filename):
        return any(filename.endswith(extension) for extension in IMG_EXTENSIONS)

    def make_dataset(self, img_root):
        images = []
        for root, _, fnames in sorted(os.walk(img_root)):
            for fname in fnames:
                if self.is_image_file(fname):
                    img_path = os.path.join(img_root, fname)
                    images.append(img_path)
        return images

In [0]:
fuse_scheme = 0
if fuse_scheme == 0:
    model_name = 'IFCNN-MAX'
elif fuse_scheme == 1:
    model_name = 'IFCNN-SUM'
elif fuse_scheme == 2:
    model_name = 'IFCNN-MEAN'
else:
    model_name = 'IFCNN-MAX'

# load pretrained model
model = myIFCNN(fuse_scheme=fuse_scheme)
model.load_state_dict(torch.load('IFCNN/Code/snapshots/IFCNN-MAX.pth'))
model.eval()
model = model.cuda()

Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /root/.cache/torch/checkpoints/resnet101-5d3b4d8f.pth


In [0]:
model

IFCNN(
  (conv2): ConvBlock(
    (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
  )
  (conv3): ConvBlock(
    (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
  )
  (conv4): Conv2d(64, 3, kernel_size=(1, 1), stride=(1, 1))
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=1, bias=False)
)

In [0]:
root = '/content/wt_cropped_data_720x720/wt_cropped_data_720x720/content/cropped_data_720x720'
n, count = get_idx_and_num_samples(root)

In [0]:
is_save = True             # Whether to save the results
is_gray = False            # Color (False) or Gray (True)
is_folder=False            # one parameter in ImageSequence
mean = [0.485, 0.456, 0.406] # Color (False) or Gray (True)
std = [0.229, 0.224, 0.225]

# get indexes and number of samples for each example
root = '/content/wt_cropped_data_720x720/wt_cropped_data_720x720/content/cropped_data_720x720'
n, count = get_idx_and_num_samples(root)

# make a folder
os.makedirs('results_max_wt')

for ind, number in zip(n, count):
    if number >= 2:
        # load the sequential source images
        filename = ind
        root = '/content/wt_cropped_data_720x720/wt_cropped_data_720x720/content/cropped_data_720x720/'
        paths = []

        if number == 2:
            paths.append(os.path.join(root, '{0}_1.png'.format(filename)))
            paths.append(os.path.join(root, '{0}_2.png'.format(filename)))

        elif number == 3:
            paths.append(os.path.join(root, '{0}_1.png'.format(filename)))
            paths.append(os.path.join(root, '{0}_2.png'.format(filename)))
            paths.append(os.path.join(root, '{0}_3.png'.format(filename)))

        elif number == 4:
            paths.append(os.path.join(root, '{0}_1.png'.format(filename)))
            paths.append(os.path.join(root, '{0}_2.png'.format(filename)))
            paths.append(os.path.join(root, '{0}_3.png'.format(filename)))            
            paths.append(os.path.join(root, '{0}_4.png'.format(filename)))

        elif number == 5:
            paths.append(os.path.join(root, '{0}_1.png'.format(filename)))
            paths.append(os.path.join(root, '{0}_2.png'.format(filename)))
            paths.append(os.path.join(root, '{0}_3.png'.format(filename)))            
            paths.append(os.path.join(root, '{0}_4.png'.format(filename)))
            paths.append(os.path.join(root, '{0}_5.png'.format(filename)))

        else:
            print("SOMETHING WROOOONG!")  

        # print(paths)
        filename = model_name + '-' + str(ind)

        seq_loader = ImageSequence(is_folder, 'RGB', transforms.Compose([
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=mean, std=std)]), *paths)
        
        imgs = seq_loader.get_imseq()
        # # perform image fusion
        with torch.no_grad():
            vimgs = []
            for idx, img in enumerate(imgs):
                img.unsqueeze_(0)
                # print(img.shape)
                vimgs.append(Variable(img.cuda()))

            vres = model(*vimgs)
            res = denorm(mean, std, vres[0]).clamp(0, 1) * 255
            res_img = res.cpu().data.numpy().astype('uint8')
            img = Image.fromarray(res_img.transpose([1, 2, 0]))
        
        # save the fused image
        if is_save:
            if is_gray:
                img.convert('L').save('results_max_wt/'+ filename+'.png', format='PNG', compress_level=0)
            else:
                img.save('results_max_wt/'+filename+'.png', format='PNG', compress_level=0)

# Metrics calculation

# brisque

In [0]:
pip install image-quality

     |████████████████████████████████| 153kB 2.8MB/s 


In [0]:
print("IFCNN-MEAN")
print("==========================")
cnn_results_mean = []
algorithm_results_mean = []

for ind in n:
  print(f'/content/results_max/IFCNN-MEAN-{ind}.png')
  print(f'/content/content/cropped_gt/{ind}_gt.png')

  img_cnn = PIL.Image.open(f'/content/results_mean/IFCNN-MEAN-{ind}.png')
  score_cnn_img = brisque.score(img_cnn)
  img_gt = PIL.Image.open(f'/content/content/cropped_gt_720x720/{ind}_gt.png')
  score_alg_img = brisque.score(img_gt)

  cnn_results_mean.append(score_cnn_img)
  algorithm_results_mean.append(score_alg_img)
  
  print(f"score of cnn image: {np.round(score_cnn_img, 3)} | score of algorithm image: {np.round(score_alg_img, 3)}")
  print("=================================================================")

IFCNN-MEAN
/content/results_max/IFCNN-MEAN-7.png
/content/content/cropped_gt/7_gt.png
score of cnn image: 29.61 | score of algorithm image: 33.088
/content/results_max/IFCNN-MEAN-30.png
/content/content/cropped_gt/30_gt.png
score of cnn image: 61.893 | score of algorithm image: 48.595
/content/results_max/IFCNN-MEAN-88.png
/content/content/cropped_gt/88_gt.png
score of cnn image: 55.023 | score of algorithm image: 49.152
/content/results_max/IFCNN-MEAN-143.png
/content/content/cropped_gt/143_gt.png
score of cnn image: 31.72 | score of algorithm image: 32.324
/content/results_max/IFCNN-MEAN-146.png
/content/content/cropped_gt/146_gt.png
score of cnn image: 35.522 | score of algorithm image: 30.788
/content/results_max/IFCNN-MEAN-27.png
/content/content/cropped_gt/27_gt.png
score of cnn image: 20.997 | score of algorithm image: 21.959
/content/results_max/IFCNN-MEAN-101.png
/content/content/cropped_gt/101_gt.png
score of cnn image: 38.709 | score of algorithm image: 37.333
/content/resul

In [0]:
print(f"mean result of cnn_max scores: {np.round(np.mean(cnn_results_mean), 3)} | mean result of algorithm scores: {np.round(np.mean(algorithm_results_mean), 3)}")

mean result of cnn_max scores: 34.907 | mean result of algorithm scores: 31.829


In [0]:
!zip -r /content/results_max_wt.zip /content/results_max_wt

  adding: content/results_max_wt/ (stored 0%)
  adding: content/results_max_wt/IFCNN-MAX-149.png (deflated 56%)
  adding: content/results_max_wt/IFCNN-MAX-280.png (deflated 44%)
  adding: content/results_max_wt/IFCNN-MAX-287.png (deflated 57%)
  adding: content/results_max_wt/IFCNN-MAX-30.png (deflated 50%)
  adding: content/results_max_wt/IFCNN-MAX-26.png (deflated 54%)
  adding: content/results_max_wt/IFCNN-MAX-285.png (deflated 49%)
  adding: content/results_max_wt/IFCNN-MAX-131.png (deflated 51%)
  adding: content/results_max_wt/IFCNN-MAX-96.png (deflated 66%)
  adding: content/results_max_wt/IFCNN-MAX-281.png (deflated 45%)
  adding: content/results_max_wt/IFCNN-MAX-16.png (deflated 57%)
  adding: content/results_max_wt/IFCNN-MAX-106.png (deflated 54%)
  adding: content/results_max_wt/IFCNN-MAX-143.png (deflated 57%)
  adding: content/results_max_wt/IFCNN-MAX-145.png (deflated 52%)
  adding: content/results_max_wt/IFCNN-MAX-87.png (deflated 50%)
  adding: content/results_max_wt/IF

In [0]:
print("IFCNN-MAX")
print("==========================")
cnn_results_max = []
algorithm_results_max = []

for ind in n:
  print(f'/content/results_max/IFCNN-MAX-{ind}.png')
  print(f'/content/content/cropped_gt/{ind}_gt.png')

  img_cnn = PIL.Image.open(f'/content/results_max/IFCNN-MAX-{ind}.png')
  score_cnn_img = brisque.score(img_cnn)
  img_gt = PIL.Image.open(f'/content/content/cropped_gt_720x720/{ind}_gt.png')
  score_alg_img = brisque.score(img_gt)

  cnn_results_max.append(score_cnn_img)
  algorithm_results_max.append(score_alg_img)
  
  print(f"score of cnn image: {np.round(score_cnn_img, 3)} | score of algorithm image: {np.round(score_alg_img, 3)}")
  print("=================================================================")

IFCNN-MAX
/content/results_max/IFCNN-MAX-7.png
/content/content/cropped_gt/7_gt.png
score of cnn image: 38.381 | score of algorithm image: 33.088
/content/results_max/IFCNN-MAX-30.png
/content/content/cropped_gt/30_gt.png
score of cnn image: 61.921 | score of algorithm image: 48.595
/content/results_max/IFCNN-MAX-88.png
/content/content/cropped_gt/88_gt.png
score of cnn image: 55.758 | score of algorithm image: 49.152
/content/results_max/IFCNN-MAX-143.png
/content/content/cropped_gt/143_gt.png
score of cnn image: 32.572 | score of algorithm image: 32.324
/content/results_max/IFCNN-MAX-146.png
/content/content/cropped_gt/146_gt.png
score of cnn image: 38.854 | score of algorithm image: 30.788
/content/results_max/IFCNN-MAX-27.png
/content/content/cropped_gt/27_gt.png
score of cnn image: 15.033 | score of algorithm image: 21.959
/content/results_max/IFCNN-MAX-101.png
/content/content/cropped_gt/101_gt.png
score of cnn image: 41.309 | score of algorithm image: 37.333
/content/results_max

In [0]:
print(f"mean result of cnn_max scores: {np.round(np.mean(cnn_results_max), 3)} | mean result of algorithm scores: {np.round(np.mean(algorithm_results_max), 3)}")

mean result of cnn_max scores: 36.19 | mean result of algorithm scores: 31.829


In [0]:
import imquality.brisque as brisque
import PIL.Image

path = '/content/results_max/IFCNN-MAX-100.png'
img = PIL.Image.open(path)
brisque.score(img)

30.011358732522893

In [0]:
path = '/content/content/cropped_gt_720x720/100_gt.png'
img = PIL.Image.open(path)
brisque.score(img)

34.66458108691879

In [0]:
import cv2
from libsvm import svmutil
from brisque import BRISQUE

In [0]:
brisq = BRISQUE()
print('Reference image: %s' % brisq.get_score('/content/results_max/IFCNN-MAX-0.png'))

Reference image: 54.16442500349808


In [0]:
print('Reference image: %s' % brisq.get_score('/content/content/cropped_gt_720x720/0_gt.png'))

Reference image: 42.18673203865626


In [0]:
!zip -r /content/results.zip /content/IFCNN/Code/results
# !cp /content/results_max.zip /content/drive/My\ Drive/

  adding: content/IFCNN/Code/results/ (stored 0%)
  adding: content/IFCNN/Code/results/IFCNN-MAX-CMF-lytro-16.png (deflated 49%)
  adding: content/IFCNN/Code/results/IFCNN-MEAN-ME-door.png (deflated 30%)
  adding: content/IFCNN/Code/results/IFCNN-MAX-IV-Camp.png (deflated 42%)
  adding: content/IFCNN/Code/results/IFCNN-MAX-MD-c04.png (deflated 63%)
  adding: content/IFCNN/Code/results/IFCNN-MEAN-ME-window.png (deflated 41%)
  adding: content/IFCNN/Code/results/IFCNN-MAX-CMF-lytro-02.png (deflated 51%)
  adding: content/IFCNN/Code/results/IFCNN-MAX-CMF-lytro-10.png (deflated 26%)
  adding: content/IFCNN/Code/results/IFCNN-MAX-IV-Trees4917.png (deflated 45%)
  adding: content/IFCNN/Code/results/IFCNN-MAX-CMF3-lytro-03.png (deflated 46%)
  adding: content/IFCNN/Code/results/IFCNN-MAX-CMF-lytro-13.png (deflated 31%)
  adding: content/IFCNN/Code/results/IFCNN-MEAN-ME-house.png (deflated 38%)
  adding: content/IFCNN/Code/results/IFCNN-MAX-IV-Road.png (deflated 51%)
  adding: content/IFCNN/Co

# CNNIQA metric

In [0]:
from argparse import ArgumentParser
import torch
from torch import nn
import torch.nn.functional as F
from PIL import Image
from CNNIQA.IQADataset import NonOverlappingCropPatches


class CNNIQAnet(nn.Module):
    def __init__(self, ker_size=7, n_kers=50, n1_nodes=800, n2_nodes=800):
        super(CNNIQAnet, self).__init__()
        self.conv1  = nn.Conv2d(1, n_kers, ker_size)
        self.fc1    = nn.Linear(2 * n_kers, n1_nodes)
        self.fc2    = nn.Linear(n1_nodes, n2_nodes)
        self.fc3    = nn.Linear(n2_nodes, 1)

    def forward(self, x):
        x  = x.view(-1, x.size(-3), x.size(-2), x.size(-1))  #

        h  = self.conv1(x)

        # h1 = F.adaptive_max_pool2d(h, 1)
        # h2 = -F.adaptive_max_pool2d(-h, 1)
        h1 = F.max_pool2d(h, (h.size(-2), h.size(-1)))
        h2 = -F.max_pool2d(-h, (h.size(-2), h.size(-1)))
        h  = torch.cat((h1, h2), 1)  # max-min pooling
        h  = h.squeeze(3).squeeze(2)

        h  = F.relu(self.fc1(h))
        h  = F.dropout(h)
        h  = F.relu(self.fc2(h))

        q  = self.fc3(h)
        return q

ModuleNotFoundError: ignored

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CNNIQAnet(ker_size=7,
                  n_kers=50,
                  n1_nodes=800,
                  n2_nodes=800).to(device)

model.load_state_dict(torch.load('/content/CNNIQA/models/CNNIQA-LIVE'))

In [0]:
def get_score(model, img_path):
    im = Image.open(img_path).convert('L')
    patches = NonOverlappingCropPatches(im, 32, 32)

    model.eval()
    with torch.no_grad():
        patch_scores = model(torch.stack(patches).to(device))
        # print(patch_scores.mean().item())

    return patch_scores.mean().item()

In [0]:
# # cnn_results = []
# # algorithm_results = []

# for ind, number in zip(n, count):
#   if number == 2:
#       # print(f'/content/results/IFCNN-MAX-{ind}.png')
#       # print(f'/content/content/cropped_gt/{ind}_gt.png')
#       score_sample1_img = get_score(model, f'/content/content/cropped_data/{ind}_1.png')
#       score_sample2_img = get_score(model, f'/content/content/cropped_data/{ind}_2.png')

#       score_alg_img = get_score(model, f'/content/content/cropped_gt/{ind}_gt.png')

#       # cnn_results.append(score_cnn_img)
#       # algorithm_results.append(score_alg_img)

#       print(f"score of sample1 image: {np.round(score_sample1_img, 3)} | score of sample2 image: {np.round(score_sample2_img, 3)} | score of algorithm image: {np.round(score_alg_img, 3)}")
#       print("=================================================================")

#   elif number == 3:
#       score_sample1_img = get_score(model, f'/content/content/cropped_data/{ind}_1.png')
#       score_sample2_img = get_score(model, f'/content/content/cropped_data/{ind}_2.png')
#       score_sample3_img = get_score(model, f'/content/content/cropped_data/{ind}_3.png')

#       score_alg_img = get_score(model, f'/content/content/cropped_gt/{ind}_gt.png')

#       # cnn_results.append(score_cnn_img)
#       # algorithm_results.append(score_alg_img)

#       print(f"score of sample1 image: {np.round(score_sample1_img, 3)} | score of sample2 image: {np.round(score_sample2_img, 3)} | score of sample3 image: {np.round(score_sample3_img, 3)} | score of algorithm image: {np.round(score_alg_img, 3)}")
#       print("=================================================================")

#   elif number == 4:
#       score_sample1_img = get_score(model, f'/content/content/cropped_data/{ind}_1.png')
#       score_sample2_img = get_score(model, f'/content/content/cropped_data/{ind}_2.png')
#       score_sample3_img = get_score(model, f'/content/content/cropped_data/{ind}_3.png')
#       score_sample4_img = get_score(model, f'/content/content/cropped_data/{ind}_4.png')

#       score_alg_img = get_score(model, f'/content/content/cropped_gt/{ind}_gt.png')

#       # cnn_results.append(score_cnn_img)
#       # algorithm_results.append(score_alg_img)

#       print(f"score of sample1 image: {np.round(score_sample1_img, 3)} | score of sample2 image: {np.round(score_sample2_img, 3)} | score of sample3 image: {np.round(score_sample3_img, 3)} | score of sample4 image: {np.round(score_sample4_img, 3)} | score of algorithm image: {np.round(score_alg_img, 3)}")
#       print("=================================================================")

#   elif number == 5:
#       score_sample1_img = get_score(model, f'/content/content/cropped_data/{ind}_1.png')
#       score_sample2_img = get_score(model, f'/content/content/cropped_data/{ind}_2.png')
#       score_sample3_img = get_score(model, f'/content/content/cropped_data/{ind}_3.png')
#       score_sample4_img = get_score(model, f'/content/content/cropped_data/{ind}_4.png')
#       score_sample5_img = get_score(model, f'/content/content/cropped_data/{ind}_5.png')

#       score_alg_img = get_score(model, f'/content/content/cropped_gt/{ind}_gt.png')

#       # cnn_results.append(score_cnn_img)
#       # algorithm_results.append(score_alg_img)

#       print(f"score of sample1 image: {np.round(score_sample1_img, 3)} | score of sample2 image: {np.round(score_sample2_img, 3)} | score of sample3 image: {np.round(score_sample3_img, 3)} | score of sample4 image: {np.round(score_sample4_img, 3)} | score of sample5 image: {np.round(score_sample5_img, 3)} | score of algorithm image: {np.round(score_alg_img, 3)}")
#       print("=================================================================")

In [0]:
print("IFCNN-MAX")
print("==========================")
cnn_results_max = []
algorithm_results_max = []

for ind in n:
  print(f'/content/results_max/IFCNN-MAX-{ind}.png')
  print(f'/content/content/cropped_gt/{ind}_gt.png')
  score_cnn_img = get_score(model, f'/content/results_max/IFCNN-MAX-{ind}.png')
  score_alg_img = get_score(model, f'/content/content/cropped_gt/{ind}_gt.png')

  cnn_results_max.append(score_cnn_img)
  algorithm_results_max.append(score_alg_img)
  
  print(f"score of cnn image: {np.round(score_cnn_img, 3)} | score of algorithm image: {np.round(score_alg_img, 3)}")
  print("=================================================================")

print("IFCNN-SUM")
print("==========================")
cnn_results_sum = []
algorithm_results_sum = []

for ind in n:
  print(f'/content/results_sum/IFCNN-SUM-{ind}.png')
  print(f'/content/content/cropped_gt/{ind}_gt.png')
  score_cnn_img = get_score(model, f'/content/results_sum/IFCNN-SUM-{ind}.png')
  score_alg_img = get_score(model, f'/content/content/cropped_gt/{ind}_gt.png')

  cnn_results_sum.append(score_cnn_img)
  algorithm_results_sum.append(score_alg_img)
  
  print(f"score of cnn image: {np.round(score_cnn_img, 3)} | score of algorithm image: {np.round(score_alg_img, 3)}")
  print("=================================================================")


print("IFCNN-MEAN")
print("==========================")
cnn_results_mean = []
algorithm_results_mean = []

for ind in n:
  print(f'/content/results_mean/IFCNN-MEAN-{ind}.png')
  print(f'/content/content/cropped_gt/{ind}_gt.png')
  score_cnn_img = get_score(model, f'/content/results_mean/IFCNN-MEAN-{ind}.png')
  score_alg_img = get_score(model, f'/content/content/cropped_gt/{ind}_gt.png')

  cnn_results_mean.append(score_cnn_img)
  algorithm_results_mean.append(score_alg_img)
  
  print(f"score of cnn image: {np.round(score_cnn_img, 3)} | score of algorithm image: {np.round(score_alg_img, 3)}")
  print("=================================================================")

IFCNN-MAX
/content/results_max/IFCNN-MAX-114.png
/content/content/cropped_gt/114_gt.png
score of cnn image: 28.565 | score of algorithm image: 25.595
/content/results_max/IFCNN-MAX-116.png
/content/content/cropped_gt/116_gt.png
score of cnn image: 27.646 | score of algorithm image: 22.683
/content/results_max/IFCNN-MAX-74.png
/content/content/cropped_gt/74_gt.png
score of cnn image: 27.602 | score of algorithm image: 24.301
/content/results_max/IFCNN-MAX-109.png
/content/content/cropped_gt/109_gt.png
score of cnn image: 24.657 | score of algorithm image: 23.437
/content/results_max/IFCNN-MAX-147.png
/content/content/cropped_gt/147_gt.png
score of cnn image: 28.206 | score of algorithm image: 23.287
/content/results_max/IFCNN-MAX-127.png
/content/content/cropped_gt/127_gt.png
score of cnn image: 26.307 | score of algorithm image: 23.829
/content/results_max/IFCNN-MAX-92.png
/content/content/cropped_gt/92_gt.png
score of cnn image: 23.272 | score of algorithm image: 23.146
/content/resul

In [0]:
print(f"mean result of cnn_max scores: {np.round(np.mean(cnn_results_max), 3)} | mean result of algorithm scores: {np.round(np.mean(algorithm_results_max), 3)}")

print(f"mean result of cnn_mean scores: {np.round(np.mean(cnn_results_mean), 3)} | mean result of algorithm scores: {np.round(np.mean(algorithm_results_mean), 3)}")

print(f"mean result of cnn_sum scores: {np.round(np.mean(cnn_results_sum), 3)} | mean result of algorithm scores: {np.round(np.mean(algorithm_results_sum), 3)}")

mean result of cnn_max scores: 25.974 | mean result of algorithm scores: 23.462
mean result of cnn_mean scores: 25.821 | mean result of algorithm scores: 23.466
mean result of cnn_sum scores: 36.543 | mean result of algorithm scores: 23.487


In [0]:
### to crop images to get rid of the black parts

# from PIL import Image
# import matplotlib.pyplot as plt

# os.makedirs('cropped_data_720x720')
# os.makedirs('cropped_gt_720x720')

# def img_crop(path, dir):
#   for file in os.walk(path):
#     for item in file[2]:
#         im = Image.open(path + "/" + item)
#         width, height = im.size
#         new_width, new_height = 750, 750

#         left = (width - new_width + 60) / 2
#         upper = (height - new_height + 20) / 2
#         right = (width + new_width) / 2
#         lower = (height + new_height - 40) / 2

#         im_cropped = im.crop((left, upper, right,lower)) # 720x720

#         im_cropped.save(dir + "/" + item, format='PNG', compress_level=0)


# img_crop("/content/content/scrapped", "/content/cropped_data_720x720")
# img_crop("/content/content/gt", "/content/cropped_gt_720x720")

# !rm -rf /content/cropped_data_720x720
# !rm -rf /content/cropped_gt_720x720